<a href="https://colab.research.google.com/github/Dharin-Dave/Code_Parser/blob/main/Code_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Summarizer**

In [ ]:
!pip install requests nbformat

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
!pip install gpt4all faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.8 MB/s eta 0:00:00


In [ ]:
!pip install nbconvert

In [ ]:
import requests
import re
import ast
from openai import OpenAI

client = OpenAI(api_key="sk-1f47c25927f44f4d95c15681d15b430a", base_url="https://api.deepseek.com/v1")

def fetch_repository_files(repository_url):
    try:
        # Extract owner and repository names from the URL
        match = re.match(r"https://github.com/([^/]+)/([^/]+)", repository_url)
        if not match:
            print("Invalid repository URL format. Please provide a valid GitHub repository URL.")
            return {}

        owner, repo_name = match.groups()

        # Get repository contents using GitHub API
        api_url = f"https://api.github.com/repos/{owner}/{repo_name}/contents"
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        contents = response.json()

        # Filter out media files and fetch contents
        files = {}
        for file_info in contents:
            if '.' in file_info['name'] and not file_info['name'].endswith(('.png', '.jpg', '.jpeg', '.gif', '.mp4', '.avi', '.mov', '.webp', '.mkv', '.md')):
                file_url = file_info['download_url']
                file_content = requests.get(file_url).text
                files[file_info['name']] = file_content

        return files
    except requests.exceptions.RequestException as e:
        print("Error fetching repository files:", e)
        return {}

# GitHub repository URL
repository_url = input('Enter repository URL: ')

# Fetch repository files and their contents
repository_files_contents = fetch_repository_files(repository_url)

for file_name, file_content in repository_files_contents.items():
# Generate AST from notebook code
  ast_tree = generate_ast(file_content)
  # Convert AST to string
  ast_str = ast.dump(ast_tree)
# Constructing user's content with the AST string
user_content = ast_str

response = client.chat.completions.create(
    model="deepseek-coder",
    messages=[
        {"role": "system", "content": '''You are a helpful assistant. Your task is to analyze and summarize the purpose and functionality
         of each function within the AST provided.'''},
        {"role": "user", "content": user_content},  # Include user's AST as content
    ]
)

print(response.choices[0].message.content)

Enter repository URL: https://github.com/AshirwadK24/Algorithms
This `Module` object encapsulates a series of cells in a Jupyter notebook. Each cell is represented as an `Expr` object with a `Dict` as its `value`, which in turn contains various elements like `Constant`, `List`, and `Dict` objects. 

Below is a detailed analysis of each function within the module and their purposes:

1. **find_heu(cur_pos, goal_pos):**
   This function calculates the heuristic cost in the A* algorithm. It uses the Euclidean distance between the current position `cur_pos` and the goal position `goal_pos` as the heuristic. The Euclidean distance is calculated using `math.sqrt`.

2. **find_neighs(cur_pos, map, closed_list, open_list, goal_pos):**
   This function finds all the valid neighbours of the current node `cur_pos` in all eight directions. Valid neighbours are those within the boundaries of the map and not in the closed list and whose map value is 0, indicating they are unobstructed. The valid neig

In [ ]:
import requests
import re

def fetch_repository_files(repository_url):
    try:
        # Extract owner and repository names from the URL
        match = re.match(r"https://github.com/([^/]+)/([^/]+)", repository_url)
        if not match:
            print("Invalid repository URL format. Please provide a valid GitHub repository URL.")
            return {}

        owner, repo_name = match.groups()

        # Get repository contents using GitHub API
        api_url = f"https://api.github.com/repos/{owner}/{repo_name}/contents"
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        contents = response.json()

        # Filter out media files and fetch contents
        files = {}
        for file_info in contents:
            if '.' in file_info['name'] and not file_info['name'].endswith(('.png', '.jpg', '.jpeg', '.gif', '.mp4', '.avi', '.mov', '.webp', '.mkv', '.md')):
                file_url = file_info['download_url']
                file_content = requests.get(file_url).text
                files[file_info['name']] = file_content

        return files
    except requests.exceptions.RequestException as e:
        print("Error fetching repository files:", e)
        return {}

# GitHub repository URL
repository_url = 'https://github.com/Prathmesh-Kolekar/ARTIMAS'

# Fetch repository files and their contents
repository_files_contents = fetch_repository_files(repository_url)

for file_name, file_content in repository_files_contents.items():
  print(file_name)
  print(file_content)

index.html
<!DOCTYPE html>
<html>

<head>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
    <meta name="viewport">
    <!--GOOGLE FONTS-->
    <!-- <link rel="preconnect" href="https://fonts.gstatic.com">
    <link href="https://fonts.googleapis.com/css2?family=Fredoka+One&family=Play&display=swap" rel="stylesheet">  -->
    <title>ARTIMAS</title>
    <link rel="icon" type="image/x-icon" href="Artimas fav.png">

    <style>
        html {
            scroll-behavior: smooth;
        }

        /* The sticky class is added to the navbar with JS when it reaches its scroll position */
        #navbar {

            height: 100vh;
            background: none;
            animation: red_art 2s 3.5s forwards linear;
            /* position: sticky; */
            top: 0;
            left: 0;
            margin: 10px;
        }

        @keyframes red_art {
            0% {
                height: 100vh;
            }

 

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
output_filename = 'output.txt'
with open(output_filename, "w") as output_file:
    for file_name, file_content in repository_files_contents.items():
      output_file.write(f"{file_name}: {file_content}\n\n")
      # Separator between files

documents = TextLoader('output.txt').load()
docs = text_splitter.split_documents(documents)
embeddings = GPT4AllEmbeddings()
db = FAISS.from_documents(docs, embeddings)
query = "Neurodrain.jpeg"
query_embedding = embeddings.embed_query(query)
result = db.similarity_search(query)
print(result[0].page_content)

'_blank')"><span><a
                                                    href="https://drive.google.com/file/d/1ApFzE6hM1JZk4A-hUZ_4i998NGGZfUyF/view?usp=sharing"
                                                    target="_blank"></a>RULEBOOK</span></button>
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>

                    <!-- quiz -->
                    <div class="col-md-3 col-sm-6">
                        <div class="our-team">
                            <div class="pic">
                                <img src="Neurodrain.jpeg">
                                <div class="over-layer">
                                    <div class="frame">
                                        <h1>NEURO DRAIN</h1>
                                        <p>Gear up for "NeuroDrain" cause it's time for BIG BRAIN.Challenge yourself
                                   

In [ ]:
import requests
import re
import ast
import nbformat
from nbconvert import PythonExporter
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key="sk-1f47c25927f44f4d95c15681d15b430a", base_url="https://api.deepseek.com/v1")

def fetch_repository_files(repository_url):
    try:
        # Extract owner, repository, and branch names from the URL
        match = re.match(r"https://github.com/([^/]+)/([^/]+)/tree/([^/]+)/(.+)", repository_url)
        if not match:
            print("Invalid repository URL format. Please provide a valid GitHub repository URL.")
            return {}

        owner, repo_name, branch, path = match.groups()

        # Get repository contents using GitHub API
        api_url = f"https://api.github.com/repos/{owner}/{repo_name}/contents/{path}?ref={branch}"
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        contents = response.json()

        # Filter out notebook files and fetch contents
        files = {}
        for file_info in contents:
            if file_info['type'] == 'file' and file_info['name'].endswith('.ipynb'):
                file_url = file_info['download_url']
                file_content = requests.get(file_url).text
                files[file_info['name']] = file_content

        return files
    except requests.exceptions.RequestException as e:
        print("Error fetching repository files:", e)
        return {}

def generate_ast_from_ipynb(file_content):
    # Convert notebook content to Python code
    notebook = nbformat.reads(file_content, as_version=4)
    exporter = PythonExporter()
    python_code, _ = exporter.from_notebook_node(notebook)

    # Parse Python code into AST
    return ast.parse(python_code)

# GitHub repository URL
repository_url = input('Enter repository URL: ')

# Fetch repository files and their contents
repository_files_contents = fetch_repository_files(repository_url)

for file_name, file_content in repository_files_contents.items():
    # Generate AST from notebook code
    ast_tree = generate_ast_from_ipynb(file_content)
    # Convert AST to string
    ast_str = ast.dump(ast_tree)
    # Split AST string into smaller chunks
    ast_chunks = [ast_str[i:i+16000] for i in range(0, len(ast_str), 16000)]

    for chunk in ast_chunks:
        # Constructing user's content with the AST string
        user_content = chunk

        response = client.chat.completions.create(
            model="deepseek-coder",
            messages=[
                {"role": "system", "content": '''You are a helpful assistant. Your task is to analyze and summarize the purpose and functionality
                 of each function within the AST provided.'''},
                {"role": "user", "content": user_content},  # Include user's AST chunk as content
            ]
        )

        print(response.choices[0].message.content)

Enter repository URL: https://github.com/openai/whisper/tree/main/notebooks
This is an abstract syntax tree (AST) representation of a Python script. Here is the broad purpose and functionality analysis:

1. It installs two Python packages: `whisper` (an automatic speech recognition system) and `jiwer` (an error rate function).
2. It imports necessary modules.
3. It attempts to import `tensorflow`, but if `ImportError` occurs, it handles the exception with a `Pass` statement.
4. It defines a class `LibriSpeech`. This class represents a PyTorch dataset for the LibriSpeech dataset. It loads the dataset, pads or trims audio to 30 seconds, and has methods for getting the dataset's length and loading specific items.
5. The script initializes a `LibriSpeech` object and a `DataLoader` from PyTorch.
6. It loads a Whisper model and defines its decoding options.
7. It prepares lists for storing hypotheses and references.
8. It then iterates through the data loader, decodes audio with the Whisper 